In [18]:
import myutils
import json
import csv
import sys
from transformers import tokenization_utils


def is_split(char):
    return tokenization_utils._is_whitespace(char) or tokenization_utils._is_punctuation(char)

csv.field_size_limit(sys.maxsize)

def write(data, path):
    outfile = open(path,  'w')
    for sent in data:
        for line in sent:
            outfile.write('\t'.join(line) + '\n')
        outfile.write('\n')
    outfile.close()

def fix_bio(data, column):
    for rowIdx in reversed(range(1,len(data))):
        if data[rowIdx][column] != 'O':
            if data[rowIdx][column] != data[rowIdx-1][column]:
                data[rowIdx][column] = 'B-' + data[rowIdx][column]
            else:
                data[rowIdx][column] = 'I-' + data[rowIdx][column]
    if data[0][column] != 'O':
        data[0][column] = 'B-' + data[0][column]
    return data

def convert(in_path, out_path):
    trainFile = in_path
    data = []
    with open(trainFile) as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        next(reader)
        data = []
        for rowIdx, row in enumerate(reader):
            subreddit_id = row[1]
            text = row[-1]
            post_id = row[0]
            subreddit_id = row[1]
            if '[deleted' in text or '[remove' in text:
                continue
            #print(text[:100])

            if 'test' in in_path:
                annotations = {}
            else:
                annotations = json.loads(row[-2])[0]['crowd-entity-annotation']['entities']


            annotations_skip = []
            skip = False

            for annotation in annotations:
                ann_beg = annotation['startOffset']
                ann_end = annotation['endOffset']
                ann_txt = text[ann_beg:ann_end]
                if ann_txt in ['[deleted]', '']:
                    annotations_skip.append(True)
                elif ann_end > len(text) + 1:
                    annotations_skip.append(True)
                else:
                    if ann_beg != 0 and not is_split(text[ann_beg]) and not is_split(text[ann_beg-1]):
                        annotations_skip.append(True)
                    elif ann_end < len(text) and not is_split(text[ann_end]) and not is_split(text[ann_end+1]) and not is_split(text[ann_end-1]):
                        annotations_skip.append(True)
                    else:
                        annotations_skip.append(False)
            if True in annotations_skip:
                continue
            #    print(annotations_skip)

            #print(text)
            #print('=-=========')
            #for annotation in annotations:
            #    print(annotation)
    
            for annotation in annotations:
                begLabel = annotation['startOffset']
                endLabel = annotation['endOffset']
                #print(text[begLabel:endLabel])
            tok = text.split(' ')
            charIdx = 0
            conll = []
            #if 'st2' in in_path:
            #    for token in row[2].split(' '):
            #        if token != '':
            #            conll.append([myutils.clean_text(token), 'O', str(token.encode()), post_id, subreddit_id])
            #    conll.append(['[SEP]', 'O'])
            for wordIdx in range(len(tok)):
                label = 'O'
                beg = charIdx + 1
                end = charIdx + len(tok[wordIdx])
                for annotation in annotations:
                    begLabel = annotation['startOffset']
                    endLabel = annotation['endOffset']
                    if (end > begLabel and end <= endLabel) or (beg >= begLabel and beg < endLabel):
                        label = annotation['label']
                charIdx += 1 + len(tok[wordIdx])
                conll.append([myutils.clean_text(tok[wordIdx]), label,str( tok[wordIdx].encode()), post_id, subreddit_id] )

            #data.append(fix_bio(conll, 1))
            data.append(conll)
    #for sentence in data:
    #    for line in sentence:
    #        print('\t'.join(line))
    #    print()
        
    split = int(.8*len(data))
    print(out_path, len(data))
    if 'test' not in in_path: 
        write(data[:split], out_path + '-train')
        write(data[split:], out_path + '-dev')
    write(data, out_path + '-all')




In [3]:

convert('../st1_train_inc_text.csv', 'st1_train.conll')

convert('../st1_test_inc_text.csv', 'st1_test.conll')

st1_train.conll 4848
st1_test.conll 1251


In [19]:


def read_conll(path, ann_col):
    curSent = []
    data = []
    for line in open(path):
        if len(line) < 2:
            data.append(curSent)
            curSent = []
        else:
            tok = line.strip().split('\t')
            word = tok[0]
            ann = tok[ann_col]
            curSent.append([word, ann])
    return data

In [20]:
train_data = read_conll('st1_train.conll-train', 1)
val_data = read_conll('st1_train.conll-dev', 1)

In [21]:
len(train_data)

3878

In [22]:
class_id = {'per_exp': 0, 'claim_per_exp': 1, 'question': 2, 'claim' : 3, 'O': 4 }

In [23]:
train_data_list = []
for num, i in enumerate(train_data):
    # print(num)
    word_list = []
    tag_list = []
    for j in i:
        try:
            tag_list.append(class_id[j[1]])
            word_list.append(j[0])
        except KeyError as e:
            continue
    train_data_list.append({'word': word_list, 'tag': tag_list})
    

In [24]:
val_data_list = []
for num, i in enumerate(val_data):
    # print(num)
    word_list = []
    tag_list = []
    for j in i:
        try:
            tag_list.append(class_id[j[1]])
            word_list.append(j[0])
        except KeyError as e:
            continue
    val_data_list.append({'word': word_list, 'tag': tag_list})

In [25]:
len(train_data_list), len(val_data_list)

(3878, 970)

In [32]:
train_data_list[0]

{'word': ['How',
  'long',
  'does',
  'it',
  'take',
  'to',
  'drop',
  'UA',
  'through',
  'diet/exerciseHey',
  'guys,',
  'I',
  'had',
  'my',
  'first',
  'flare',
  'up',
  'over',
  'a',
  'week',
  'ago.',
  'The',
  'ER',
  'gave',
  'me',
  'colchicine',
  'and',
  'im',
  'doing',
  'much',
  'better',
  'now.A',
  'few',
  'days',
  'after',
  'my',
  'attack',
  'my',
  'UA',
  'was',
  '9.6.',
  'Today',
  'I',
  'received',
  'an',
  'Amazon',
  'tester',
  'and',
  "it's",
  'showing',
  '10.3.',
  'I',
  'understand',
  'that',
  'it',
  'drops',
  'during',
  'an',
  'attack,',
  'but',
  'I',
  'was',
  'surprised',
  'that',
  'it',
  'would',
  'be',
  'such',
  'a',
  "difference.I've",
  'been',
  'dieting',
  'and',
  'exercising',
  'to',
  'try',
  'and',
  'prevent',
  'a',
  'second',
  'attack',
  'while',
  'I',
  'wait',
  'for',
  'my',
  'rheumatologist',
  'appointment.',
  'My',
  'question',
  'is',
  'how',
  'long',
  'does',
  'it',
  'take',


In [29]:
data_dict = {'train': train_data_list, 'validation': val_data_list}

In [50]:
import transformers
from datasets import load_dataset, load_metric

task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 2

from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)



In [36]:
from datasets import Dataset, DatasetDict

dataset_train = Dataset.from_list(train_data_list, split='train')
dataset_validate = Dataset.from_list(val_data_list, split='validation')

data_dict = {'train': dataset_train, 'validation': dataset_validate}

print(len(dataset))

3878


In [37]:
dataset_final = DatasetDict(data_dict)

In [38]:
dataset_final

DatasetDict({
    train: Dataset({
        features: ['word', 'tag'],
        num_rows: 3878
    })
    validation: Dataset({
        features: ['word', 'tag'],
        num_rows: 970
    })
})

In [39]:
def tokenize_and_align_labels(examples,  label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["word"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tag']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [40]:
tokenized_datasets = dataset_final.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3878 [00:00<?, ? examples/s]

Map:   0%|          | 0/970 [00:00<?, ? examples/s]

In [41]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['word', 'tag', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3878
    })
    validation: Dataset({
        features: ['word', 'tag', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 970
    })
})

In [42]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import torch

label_list = ['per_exp', 'claim_per_exp', 'question', 'claim' , 'O']

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

metric = load_metric("seqeval")

/Users/23395885/Desktop/CasualClaimID/pythonProject/.venv/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/Users/23395885/Desktop/CasualClaimID/pythonProject/.venv/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [52]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [53]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [54]:
trainer.train()

RuntimeError: MPS backend out of memory (MPS allocated: 10.02 GB, other allocations: 10.35 GB, max allowed: 20.40 GB). Tried to allocate 89.42 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
trainer.evaluate()